### Goldsmiths University of London
### Authors...: Sandor Kanda (skand001) + Carlos Alves (cdeol003)
### Created...: 13/02/2023

## The Task:

The task is based on an energy efficiency prediction problem which is a regression problem. 
More specifically, you are required to perform an analysis using different building shapes, 
with different characteristics, and predict the heating load of the building. 
The buildings differ with respect to the glazing area, the glazing area distribution, the 
orientation, and other aspects comprised in the dataset.
The dataset (below) for this task includes 9 features, denoted by X0, X1, ..., X8, and an 
outcome variable Y which needs to be predicted. 

#### The dataset contains missing values. Here is the meaning of the 10 variables:

X0: Category of the building
X1: Relative Compactness
X2: Surface Area
X3: Wall Area
X4: Roof Area
X5: Overall Height
X6: Orientation
X7: Glazing Area
X8: Glazing Area Distribution

Y: Heating Load

#### You are required to perform this analysis comprising: 
#### 1. Data inspection and visualisation, 
#### 2. Data preprocessing including data splitting in 70% training data, and 30% test data, 
#### 3. Data transformations you consider useful for this task, 
#### 3.1. Treatment of missing values, 
#### 3.2. Feature selection if you consider it useful for helping you achieve a better performance, etc. 

#### The analysis should include developing the predictive models based on the following algorithms already studied in this module, or that are going to be studied such as neural networks: 
#### 1. Simple Linear Regression, 
#### 2. Ridge Regression, 
#### 3. Lasso Regression, 
#### 4. Elastic Net Regression, 
#### 5. Polynomial Regression with regularisation, 
#### 6. Neural Network. 

#### These models (except simple Linear Regression which needs only to be simply trained) will be:
#### 1. Tuned using the training set. 
#### 2. The training set will be used to select the best 2 models. 
#### 3. Only these 2 best models will be evaluated on the test set. 

#### The code, comments and explanations will be provided in a Python Jupyter notebook, which should include also the results of running your notebook.

In [1]:
# Import relevant libraries for the project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Import the dataset for the project
dataset = pd.read_csv('datcw_na.csv')